# Understanding executables

Running on an Orange Pi Zero, with 32-bit ARMv7-A Cortex-A7 architecture.
* NEON SIMD instruction set
* VFPv4 Floating Point Unit

## References

* https://stackoverflow.com/questions/200292/process-for-reducing-the-size-of-an-executable
* http://www.muppetlabs.com/~breadbox/software/tiny/
* https://stac47.github.io/c/relocation/elf/tutorial/2018/03/01/understanding-relocation-elf.html
* http://timelessname.com/elfbin/

In [11]:
! uname -a

Linux master 5.10.60-sunxi #21.08.1 SMP Wed Aug 25 18:19:32 UTC 2021 armv7l armv7l armv7l GNU/Linux


In [13]:
! lscpu

Architecture:        armv7l
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  4
Socket(s):           1
Vendor ID:           ARM
Model:               5
Model name:          Cortex-A7
Stepping:            r0p5
CPU max MHz:         1008.0000
CPU min MHz:         480.0000
BogoMIPS:            22.85
Flags:               half thumb fastmult vfp edsp neon vfpv3 tls vfpv4 idiva idivt vfpd32 lpae evtstrm


In [1]:
%%writefile helloworld.c
#include <stdio.h>
extern int puts(const char *);
void main(void)
{
    puts("Hello, world!");
}

Writing helloworld.c


In [2]:
! gcc helloworld.c -o helloworld

In [6]:
! ./helloworld

Hello, world!


In [5]:
! stat -c "%s %n" helloworld

8100 helloworld


In [9]:
! size helloworld

   text	   data	    bss	    dec	    hex	filename
   1077	    324	      4	   1405	    57d	helloworld


In [7]:
! objdump -x helloworld


helloworld:     file format elf32-littlearm
helloworld
architecture: arm, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x000003fd

Program Header:
0x70000001 off    0x00000584 vaddr 0x00000584 paddr 0x00000584 align 2**2
         filesz 0x00000008 memsz 0x00000008 flags r--
    PHDR off    0x00000034 vaddr 0x00000034 paddr 0x00000034 align 2**2
         filesz 0x00000120 memsz 0x00000120 flags r--
  INTERP off    0x00000154 vaddr 0x00000154 paddr 0x00000154 align 2**0
         filesz 0x00000019 memsz 0x00000019 flags r--
    LOAD off    0x00000000 vaddr 0x00000000 paddr 0x00000000 align 2**16
         filesz 0x00000590 memsz 0x00000590 flags r-x
    LOAD off    0x00000ec4 vaddr 0x00010ec4 paddr 0x00010ec4 align 2**16
         filesz 0x00000144 memsz 0x00000148 flags rw-
 DYNAMIC off    0x00000ecc vaddr 0x00010ecc paddr 0x00010ecc align 2**2
         filesz 0x000000f8 memsz 0x000000f8 flags rw-
    NOTE off    0x00000170 vaddr 0x00000170 paddr 0x00000170 align 2**2
      

In [8]:
! objdump -d helloworld


helloworld:     file format elf32-littlearm


Disassembly of section .init:

000003a0 <_init>:
 3a0:	e92d4008 	push	{r3, lr}
 3a4:	eb000026 	bl	444 <call_weak_fn>
 3a8:	e8bd8008 	pop	{r3, pc}

Disassembly of section .plt:

000003ac <.plt>:
 3ac:	e52de004 	push	{lr}		; (str lr, [sp, #-4]!)
 3b0:	e59fe004 	ldr	lr, [pc, #4]	; 3bc <.plt+0x10>
 3b4:	e08fe00e 	add	lr, pc, lr
 3b8:	e5bef008 	ldr	pc, [lr, #8]!
 3bc:	00010c08 	.word	0x00010c08

000003c0 <__cxa_finalize@plt>:
 3c0:	e28fc600 	add	ip, pc, #0, 12
 3c4:	e28cca10 	add	ip, ip, #16, 20	; 0x10000
 3c8:	e5bcfc08 	ldr	pc, [ip, #3080]!	; 0xc08

000003cc <puts@plt>:
 3cc:	e28fc600 	add	ip, pc, #0, 12
 3d0:	e28cca10 	add	ip, ip, #16, 20	; 0x10000
 3d4:	e5bcfc00 	ldr	pc, [ip, #3072]!	; 0xc00

000003d8 <__libc_start_main@plt>:
 3d8:	e28fc600 	add	ip, pc, #0, 12
 3dc:	e28cca10 	add	ip, ip, #16, 20	; 0x10000
 3e0:	e5bcfbf8 	ldr	pc, [ip, #3064]!	; 0xbf8

000003e4 <__gmon_start__@plt>:
 3e4:	e28fc600 	add	ip, pc, #0, 12
 3e8:	e28cca10 	add	ip, ip

In [10]:
! hexdump -C helloworld

00000000  7f 45 4c 46 01 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  03 00 28 00 01 00 00 00  fd 03 00 00 34 00 00 00  |..(.........4...|
00000020  1c 1b 00 00 00 04 00 05  34 00 20 00 09 00 28 00  |........4. ...(.|
00000030  1d 00 1c 00 01 00 00 70  84 05 00 00 84 05 00 00  |.......p........|
00000040  84 05 00 00 08 00 00 00  08 00 00 00 04 00 00 00  |................|
00000050  04 00 00 00 06 00 00 00  34 00 00 00 34 00 00 00  |........4...4...|
00000060  34 00 00 00 20 01 00 00  20 01 00 00 04 00 00 00  |4... ... .......|
00000070  04 00 00 00 03 00 00 00  54 01 00 00 54 01 00 00  |........T...T...|
00000080  54 01 00 00 19 00 00 00  19 00 00 00 04 00 00 00  |T...............|
00000090  01 00 00 00 01 00 00 00  00 00 00 00 00 00 00 00  |................|
000000a0  00 00 00 00 90 05 00 00  90 05 00 00 05 00 00 00  |................|
000000b0  00 00 01 00 01 00 00 00  c4 0e 00 00 c4 0e 01 00  |................|
000000c0  c4 0e 01 00 44 01 00 00  48 01 00 00 06 00

In [1]:
! strip --strip-all --verbose helloworld

copy from `helloworld' [elf32-littlearm] to `stI8J6gI' [elf32-littlearm]


In [2]:
! ./helloworld

Hello, world!


In [3]:
! stat -c "%s %n" helloworld    # era 8100

5528 helloworld


In [4]:
! size helloworld

   text	   data	    bss	    dec	    hex	filename
   1077	    324	      4	   1405	    57d	helloworld


In [5]:
! objdump -x helloworld


helloworld:     file format elf32-littlearm
helloworld
architecture: arm, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x000003fd

Program Header:
0x70000001 off    0x00000584 vaddr 0x00000584 paddr 0x00000584 align 2**2
         filesz 0x00000008 memsz 0x00000008 flags r--
    PHDR off    0x00000034 vaddr 0x00000034 paddr 0x00000034 align 2**2
         filesz 0x00000120 memsz 0x00000120 flags r--
  INTERP off    0x00000154 vaddr 0x00000154 paddr 0x00000154 align 2**0
         filesz 0x00000019 memsz 0x00000019 flags r--
    LOAD off    0x00000000 vaddr 0x00000000 paddr 0x00000000 align 2**16
         filesz 0x00000590 memsz 0x00000590 flags r-x
    LOAD off    0x00000ec4 vaddr 0x00010ec4 paddr 0x00010ec4 align 2**16
         filesz 0x00000144 memsz 0x00000148 flags rw-
 DYNAMIC off    0x00000ecc vaddr 0x00010ecc paddr 0x00010ecc align 2**2
         filesz 0x000000f8 memsz 0x000000f8 flags rw-
    NOTE off    0x00000170 vaddr 0x00000170 paddr 0x00000170 align 2**2
      

In [ ]:
! strip helloworld --remove-section=.comment --remove-section=.note* \
    --remove-section=.gnu*